<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="3em" style="font-weight:bold;color:#3f8dbf;">  </font><br>
    
### 주요 질문(탐색하고자 하는 질문 리스트)
- 얼마나 많은 고객이 있는가?
- 고객은 어디에 주로 사는가?
- 고객은 주로 어떤 지불방법을 사용하는가?
- **평균 거래액은 얼마일까?**
- 일별, 주별, 월별 판매 트렌드는?
- 어떤 카테고리가 가장 많은 상품이 팔렸을까?
- 평균 배송시간은?
</div>

In [1]:
import pandas as pd

customers = pd.read_csv('00_data/olist_customers_dataset.csv', encoding = 'utf-8-sig')
geolocations = pd.read_csv('00_data/olist_geolocation_dataset.csv', encoding = 'utf-8-sig')
items = pd.read_csv('00_data/olist_order_items_dataset.csv', encoding = 'utf-8-sig')
payments = pd.read_csv('00_data/olist_order_payments_dataset.csv', encoding = 'utf-8-sig')
reviews = pd.read_csv('00_data/olist_order_reviews_dataset.csv', encoding = 'utf-8-sig')
orders = pd.read_csv('00_data/olist_orders_dataset.csv', encoding = 'utf-8-sig')
sellers = pd.read_csv('00_data/olist_sellers_dataset.csv', encoding = 'utf-8-sig')
products = pd.read_csv('00_data/olist_products_dataset.csv', encoding = 'utf-8-sig')

In [2]:
payments_copy = payments.groupby('order_id').sum()

orders_payments = pd.merge(orders, payments_copy, on = 'order_id')
orders_payments['order_purchase_timestamp'] = pd.to_datetime(orders_payments['order_purchase_timestamp'], format = '%Y-%m-%d %H:%M:%S', errors = 'raise')
orders_payments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99440 entries, 0 to 99439
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99440 non-null  object        
 1   customer_id                    99440 non-null  object        
 2   order_status                   99440 non-null  object        
 3   order_purchase_timestamp       99440 non-null  datetime64[ns]
 4   order_approved_at              99280 non-null  object        
 5   order_delivered_carrier_date   97657 non-null  object        
 6   order_delivered_customer_date  96475 non-null  object        
 7   order_estimated_delivery_date  99440 non-null  object        
 8   payment_sequential             99440 non-null  int64         
 9   payment_installments           99440 non-null  int64         
 10  payment_value                  99440 non-null  float64       
dtypes: datetime64[n

In [3]:
orders_payments_date_value = orders_payments[['order_purchase_timestamp', 'payment_value']].copy()
#                                           두번
orders_payments_date_value.head()

,order_purchase_timestamp,payment_value
0,2017-10-02 10:56:33,38.71
1,2018-07-24 20:41:37,141.46
2,2018-08-08 08:38:49,179.12
3,2017-11-18 19:28:06,72.20
4,2018-02-13 21:18:39,28.62


---
- aggregate 를 쓴다면

In [ ]:
products_category_order = products_category_order.groupby('product_category_name_english').aggregate({'order_id':'count'}).rename(columns={'order_id':'order_count'})

In [ ]:
orders_payments_date_value2 = orders_payments.groupby('order_purchase_timestamp').aggregate('payment_value':'sum')

gorupby 한 상태를 merge 했기에 ㄴㄴ

In [4]:
orders_payments_by_month = orders_payments_date_value.groupby(pd.Grouper(key = 'order_purchase_timestamp', freq = 'M')).sum()
orders_payments_by_month.head()

,payment_value
order_purchase_timestamp,
2016-09-30,252.24
2016-10-31,59090.48
2016-11-30,0.00
2016-12-31,19.62
2017-01-31,138488.04


In [5]:
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected = True)

In [6]:
orders_payments_by_month.iplot(kind = 'bar')

---
- 실제 거래된 주문건 확인

In [11]:
orders_payments['order_status'].value_counts()

delivered      96477
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: order_status, dtype: int64

In [12]:
orders_payments.dropna()['order_status'].value_counts()

delivered    96454
canceled         6
Name: order_status, dtype: int64

In [13]:
orders_payments.dropna().drop(['order_status'] == 'canceled')['order_status'].value_counts()

KeyError: '[False] not found in axis'

In [20]:
type(orders_payments.dropna()[orders_payments['order_status'] == 'canceled'].index.to_list())

<ipython-input-20-1efceb6eb079>:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



list

In [22]:
drop_canceled = orders_payments.dropna()[orders_payments['order_status'] == 'canceled'].index.to_list()

<ipython-input-22-f2efd6c764d3>:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [23]:
for drops in drop_canceled:
    orders_payments = orders_payments.dropna().drop([drops])

orders_payments['order_status'].value_counts()

delivered    96454
Name: order_status, dtype: int64

In [24]:
orders_payments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96454 entries, 0 to 99439
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       96454 non-null  object        
 1   customer_id                    96454 non-null  object        
 2   order_status                   96454 non-null  object        
 3   order_purchase_timestamp       96454 non-null  datetime64[ns]
 4   order_approved_at              96454 non-null  object        
 5   order_delivered_carrier_date   96454 non-null  object        
 6   order_delivered_customer_date  96454 non-null  object        
 7   order_estimated_delivery_date  96454 non-null  object        
 8   payment_sequential             96454 non-null  int64         
 9   payment_installments           96454 non-null  int64         
 10  payment_value                  96454 non-null  float64       
dtypes: datetime64[n

In [26]:
orders_payments_delivered = orders_payments[['order_purchase_timestamp','payment_value']].copy()
orders_payments_month = orders_payments_delivered.groupby(pd.Grouper(key = 'order_purchase_timestamp', freq = 'M')).sum()

In [38]:
import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(
    go.Bar(
    x = orders_payments_month.index,
    y = orders_payments_month['payment_value'],
    text = orders_payments_month['payment_value'],
    textposition = 'auto',
    texttemplate = 'R$ %{text:.0f}'
    )
)

fig.update_layout(
    {
        'title': {
            'text': '<b>Turnover per Month in Brazilian Olist E-commerce company</b>',
            'x': 0.5,
            'y': 0.8,
            'font':{
                'size': 15
            }
        },
        'xaxis': {
            'title': 'from Oct.2016 to Sep. 2018',
            'showticklabels': True,
            'dtick': 'M1',
            'tickfont': {
                'size': 7
            }
        },
        'yaxis': {
            'title': 'Turnover per Month'
        }
    }
)

fig.show()

In [39]:
orders_payments_month_form_Feb_2017 = orders_payments_month[orders_payments_month.index > '2017-01-01']
orders_payments_month_form_Feb_2017

,payment_value
order_purchase_timestamp,
2017-01-31,127430.74
2017-02-28,269458.98
2017-03-31,414369.39
2017-04-30,390952.18
2017-05-31,566872.73
2017-06-30,490225.60
2017-07-31,566403.93
2017-08-31,646000.61
2017-09-30,700976.01


In [43]:
import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(
    go.Bar(
    x = orders_payments_month_form_Feb_2017.index,
    y = orders_payments_month_form_Feb_2017['payment_value'],
    text = orders_payments_month_form_Feb_2017['payment_value'],
    textposition = 'auto',
    texttemplate = 'R$ %{text:.0f}'
    )
)

fig.update_layout(
    {
        'title': {
            'text': '<b>Turnover per Month in Brazilian Olist E-commerce company</b>',
            'x': 0.5,
            'y': 0.8,
            'font':{
                'size': 15
            }
        },
        'xaxis': {
            'title': 'from Oct.2016 to Sep. 2018',
            'showticklabels': True,
            'tick0': '2017-01-31',
            #Feb.2017 부터 시작
            'dtick': 'M1',
            'tickfont': {
                'size': 7
            }
        },
        'yaxis': {
            'title': 'Turnover per Month'
        },
        # ,
        'template': 'plotly_white'
        #표 template도 추가
    }
)

fig.show()

---
- annotation

In [47]:
colors = ['#03588C'] * len(orders_payments_month_form_Feb_2017.index)
colors[10] = '#F24472'
#    s

In [48]:
import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(
    go.Bar(
    x = orders_payments_month_form_Feb_2017.index,
    y = orders_payments_month_form_Feb_2017['payment_value'],
    text = orders_payments_month_form_Feb_2017['payment_value'],
    textposition = 'auto',
    texttemplate = 'R$ %{text:.0f}',
    marker_color = colors
    )
)

fig.update_layout(
    {
        'title': {
            'text': '<b>Turnover per Month in Brazilian Olist E-commerce company</b>',
            'x': 0.5,
            'y': 0.8,
            'font':{
                'size': 15
            }
        },
        'xaxis': {
            'title': 'from Oct.2016 to Sep. 2018',
            'showticklabels': True,
            'tick0': '2017-01-31',
            #Feb.2017 부터 시작
            'dtick': 'M1',
            'tickfont': {
                'size': 7
            }
        },
        'yaxis': {
            'title': 'Turnover per Month'
        },
        # ,
        'template': 'plotly_white'
        #표 template도 추가
    }
)

fig.show()

In [94]:
import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(
    go.Bar(
    x = orders_payments_month_form_Feb_2017.index,
    y = orders_payments_month_form_Feb_2017['payment_value'],
    text = orders_payments_month_form_Feb_2017['payment_value'],
    textposition = 'auto',
    texttemplate = 'R$ %{text:.0f}',
    marker_color = colors
    )
)

fig.update_layout(
    {
        'title': {
            'text': '<b>Turnover per Month in Brazilian Olist E-commerce company</b>',
            'x': 0.5,
            'y': 0.9,
            'font':{
                'size': 15
            }
        },
        'xaxis': {
            'title': 'from Oct.2016 to Sep. 2018',
            'showticklabels': True,
            'tick0': '2017-01-31',
            #Feb.2017 부터 시작
            'dtick': 'M1',
            'tickfont': {
                'size': 7
            }
        },
        'yaxis': {
            'title': 'Turnover per Month'
        },
        # ,
        'template': 'plotly_white'
        #표 template도 추가
    }
)

fig.add_annotation(
    x = '2017-11-30',
    y = orders_payments_month_form_Feb_2017['payment_value'].loc['2017-11-30'],
    text = '<b>Peacked Monthly Turnover</b>',
    showarrow = True,
    font = dict(
        size = 10,
        color = '#ffffff'
    ),
    align = 'center',

    arrowhead = 2,
    arrowsize=2,
    arrowwidth =1,
    arrowcolor="#77CFD9",
    
    ax=30,
    ay=-30,
    
    bordercolor="#77CFD9",
    borderwidth=3,
    
    borderpad=8,
    bgcolor="#F25D50",
    
    opacity=0.7
)
    
fig.show()